In [11]:
import open3d as o3d
import numpy as np
import os
import cv2

### Visualize data at each timestamp

In [12]:
def get_subdirectory_names(target_dir):
    return os.listdir(target_dir)

def get_file_names(target_dir):
    return os.listdir(target_dir)

def get_timestamps(target_dir, is_sorted=True):
    filenames = [file_name.split('.')[0] for file_name in get_file_names(target_dir)]
    timestamps = list(set(filenames))
    return timestamps

def get_sorted_timestamps(target_dir):
    timestamps = get_timestamps(target_dir)
    sorted_timestamps = sorted(timestamps, key=int)
    return sorted_timestamps

SIMULATION_DATA_PATH = '/media/leppsalu/SSD_Storage/generated_data_town03_sample'
SENSOR_DATA_DIRS = get_subdirectory_names(SIMULATION_DATA_PATH) 
SENSOR_DATA_DIRS = [dir_name for dir_name in SENSOR_DATA_DIRS if dir_name == dir_name.upper()]
LIDAR_DATA_DIRS = [dir_name for dir_name in SENSOR_DATA_DIRS if 'LIDAR' in dir_name]
RADAR_DATA_DIRS = [dir_name for dir_name in SENSOR_DATA_DIRS if 'RADAR' in dir_name]
CAMERA_DATA_DIRS = [dir_name for dir_name in SENSOR_DATA_DIRS if 'CAM_' in dir_name]
DEPTH_BEV_DATA_DIR = "DEPTH_BEV"
DEPTH_VISIBILITY_DATA_DIR = "DEPTH_VISIBILITY"
TIMESTAMPS = get_sorted_timestamps(os.path.join(SIMULATION_DATA_PATH, LIDAR_DATA_DIRS[0]))

In [13]:
i = 0
n = 1000                                                                                                                                                                                                                                                                                                            
timestamp = TIMESTAMPS[i]
n_timestamps = TIMESTAMPS[i:n]

### Visualize occupancy and visibility maps together

In [14]:
for timestamp in n_timestamps:
    occupancy_grid = cv2.imread(os.path.join(SIMULATION_DATA_PATH, DEPTH_BEV_DATA_DIR, f"{timestamp}.bev.png"), cv2.IMREAD_GRAYSCALE)
    visibility_grid = cv2.imread(os.path.join(SIMULATION_DATA_PATH, DEPTH_VISIBILITY_DATA_DIR, f"{timestamp}.png"), cv2.IMREAD_GRAYSCALE)

    visibility_and_occupancy = np.zeros_like(occupancy_grid)    
    visibility_and_occupancy[visibility_grid == 255] = 255
    visibility_and_occupancy[occupancy_grid == 255] = 125  
    visibility_and_occupancy = cv2.resize(visibility_and_occupancy, (1000, 1000), interpolation=cv2.INTER_NEAREST)       

    cv2.imshow("Visibility and Occupancy", visibility_and_occupancy)
    print("Current timestamp:", timestamp)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        print("Final timestamp:", timestamp)
        break              
cv2.destroyAllWindows()      

Current timestamp: 60061680012
Current timestamp: 60161680014
Final timestamp: 60161680014


### Visualization functions

In [15]:
def add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_name=None, coordinate_frame_size=0.2):
    # Create an Open3D geometry from the transformation matrix
    frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=coordinate_frame_size, origin=[0, 0, 0])
    frame.transform(transformation_matrix)
    vis.add_geometry(frame)
    # If a name is provided, add a text label to the frame
    # if coordinate_frame_name is not None:
    #     text = o3d.t.geometry.TriangleMesh.create_text(coordinate_frame_name).to_legacy()
    #     text.translate([0, 0, 0.1])
    #     text.transform(transformation_matrix)

    #     vis.add_geometry(text)

def add_pointcloud_to_visualizer(vis, pointcloud, transformation_matrix=None, color=[1.0, 0.0, 0.0]):
    pointcloud.paint_uniform_color(color)
    if transformation_matrix is not None:
        pointcloud.transform(transformation_matrix)
    vis.add_geometry(pointcloud)

def add_all_lidar_poses_to_visualizer(vis, timestamp):
    for data_dir in LIDAR_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=3, coordinate_frame_name=data_dir)

def add_all_radar_poses_to_visualizer(vis, timestamp):
    for data_dir in RADAR_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=2, coordinate_frame_name=data_dir)

def add_all_camera_poses_to_visualizer(vis, timestamp):
    for data_dir in CAMERA_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=1.5, coordinate_frame_name=data_dir)

def add_all_sensor_poses_to_visualizer(vis, timestamp):
    for data_dir in LIDAR_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=3, coordinate_frame_name=data_dir)
    for data_dir in RADAR_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=2, coordinate_frame_name=data_dir)
    for data_dir in CAMERA_DATA_DIRS:
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_coordinate_frame_to_visualizer(vis, transformation_matrix, coordinate_frame_size=1.5, coordinate_frame_name=data_dir)

#### Visualize lidar point cloud with depth point cloud

In [16]:
cv2.namedWindow("Press q to exit visualization loop", cv2.WINDOW_NORMAL)
for i, timestamp in enumerate(n_timestamps):
    if cv2.waitKey(0) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        print("Exiting loop at timestamp:", timestamp)
        break
    else:
        vis = o3d.visualization.Visualizer()
        vis.create_window(window_name=f"Lidar point clouds (red) + Depth point clouds (green) + Sensor poses")
        lidar_pointcloud = o3d.io.read_point_cloud(os.path.join(SIMULATION_DATA_PATH, LIDAR_DATA_DIRS[0], f'{timestamp}.ply'))
        depth_pointcloud = o3d.io.read_point_cloud(os.path.join(SIMULATION_DATA_PATH, DEPTH_VISIBILITY_DATA_DIR, f'{timestamp}.ply'))
        add_pointcloud_to_visualizer(vis, lidar_pointcloud, color=[1.0, 0.0, 0.0])
        add_pointcloud_to_visualizer(vis, depth_pointcloud, color=[0.0, 1.0, 0.0])
        if i == 0:
            add_all_sensor_poses_to_visualizer(vis, timestamp)
        else:
            add_all_lidar_poses_to_visualizer(vis, timestamp)
        vis.run()
        vis.destroy_window()

    
        

Exiting loop at timestamp: 60661680021


#### Highlight filtered points in semantic point cloud

In [17]:
# from classes.CARLASemantics import SemanticTags

# def get_points_from_semantic_point_cloud(point_cloud, semantic_tags=None):
#     point_cloud_points = np.array(point_cloud.points)
#     point_cloud_colors = (np.array(point_cloud.colors) * 255).astype(np.uint8) 
#     point_cloud_semantic_tags = point_cloud_colors[:, 2]
#     if semantic_tags is not None:
#         semantic_tags = np.array(semantic_tags)
#         point_cloud_points = point_cloud_points[np.where(np.isin(point_cloud_semantic_tags, semantic_tags))]
#     return point_cloud_points

# def get_obstacles_from_semantic_point_cloud(point_cloud):
#     all_semantic_tags = [semantic_tag.value for semantic_tag in SemanticTags]
#     ground_and_sky_semantic_tags = np.array([ 
#             SemanticTags.ROADLINE.value, SemanticTags.ROAD.value, 
#             SemanticTags.SIDEWALK.value, SemanticTags.GROUND.value, 
#             SemanticTags.WATER.value, SemanticTags.TERRAIN.value
#         ], dtype=np.uint8)
#     obstacles_tags = np.setdiff1d(all_semantic_tags, ground_and_sky_semantic_tags)
#     obstacle_points = get_points_from_semantic_point_cloud(point_cloud, obstacles_tags)
#     obstacle_point_cloud = o3d.geometry.PointCloud()
#     obstacle_point_cloud.points = o3d.utility.Vector3dVector(obstacle_points)
#     return obstacle_point_cloud

# point_cloud = o3d.io.read_point_cloud(os.path.join(SIMULATION_DATA_PATH, DEPTH_VISIBILITY_DATA_DIR, "43190989933.ply"))  
# obstacles = get_obstacles_from_semantic_point_cloud(point_cloud)
# vi = o3d.visualization.Visualizer()
# vi.create_window(window_name="Obstacles")
# add_pointcloud_to_visualizer(vi, point_cloud, color=[1.0, 0.0, 0.0])
# add_pointcloud_to_visualizer(vi, obstacles, color=[0.0, 0.0, 1.0])

# vi.run()
# vi.destroy_window()
# raise RuntimeError("Stop here")

### Visualize lidar scans

In [18]:
def add_lidar_readings_to_visualizer(vis, timestamp):
    for data_dir in LIDAR_DATA_DIRS:
        pointcloud = o3d.io.read_point_cloud(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.ply'))
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_pointcloud_to_visualizer(vis, pointcloud, transformation_matrix)

In [19]:
# Visualize one frame
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Single frame: Ego Vehicle Sensors + LIDAR point clouds', width=1920, height=1080)

add_all_sensor_poses_to_visualizer(vis, timestamp)
add_lidar_readings_to_visualizer(vis, timestamp)

vis.run()
vis.destroy_window()

In [20]:
# Visualize multiple frames
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Multiple frames: Ego Vehicle Sensors + LIDAR point clouds', width=1920, height=1080)

for i, timestamp in enumerate(n_timestamps):
    if i % 90 == 0:
        add_all_sensor_poses_to_visualizer(vis, timestamp)
    else:
        add_all_lidar_poses_to_visualizer(vis, timestamp)
    add_lidar_readings_to_visualizer(vis, timestamp)

vis.run()
vis.destroy_window()
raise RuntimeError("Stop here")

RuntimeError: Stop here

### Visualize radar scans

In [ ]:
def add_radar_readings_to_visualizer(vis, timestamp):
    for data_dir in RADAR_DATA_DIRS:
        pointcloud = o3d.io.read_point_cloud(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.ply'))
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_pointcloud_to_visualizer(vis, pointcloud, transformation_matrix)

In [ ]:
# Visualize one frame
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Single frame: Ego Vehicle Sensors + RADAR point clouds', width=1920, height=1080)

add_all_sensor_poses_to_visualizer(vis, timestamp)
add_radar_readings_to_visualizer(vis, timestamp)

vis.run()
vis.destroy_window()

In [ ]:
# Visualize multiple frames
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Multiple frames: Ego Vehicle Sensors + RADAR point clouds', width=1920, height=1080)

for i, timestamp in enumerate(n_timestamps):
    if i % 90 == 0:
        add_all_sensor_poses_to_visualizer(vis, timestamp)
    else:
        add_all_lidar_poses_to_visualizer(vis, timestamp)
    add_radar_readings_to_visualizer(vis, timestamp)

vis.run()
vis.destroy_window()

### Visualize depth cam scans

In [ ]:
def add_depth_camera_readings_to_visualizer(vis, timestamp):
    for data_dir in ['DEPTH_BEV']:
        pointcloud = o3d.io.read_point_cloud(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.obstacles.ply'))
        transformation_matrix = np.load(os.path.join(SIMULATION_DATA_PATH, data_dir, f'{timestamp}.npy'))
        add_pointcloud_to_visualizer(vis, pointcloud, transformation_matrix)

In [ ]:
# Visualize one frame
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Single frame: Ego Vehicle Sensors + Depth camera point clouds', width=1920, height=1080)

add_all_sensor_poses_to_visualizer(vis, timestamp)
add_depth_camera_readings_to_visualizer(vis, timestamp)

# Run the visualization
vis.run()

# Close the visualization window
vis.destroy_window()

In [ ]:
# Visualize multiple frames
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Multiple frame: Ego Vehicle Sensors + Depth camera point clouds', width=1920, height=1080)

for i, timestamp in enumerate(n_timestamps):
    if i % 90 == 0:
        add_all_sensor_poses_to_visualizer(vis, timestamp)
    else:
        add_all_lidar_poses_to_visualizer(vis, timestamp)
    add_depth_camera_readings_to_visualizer(vis, timestamp)

vis.run()
vis.destroy_window()